In [77]:
import datasets
import tqdm
import random
from transformers import AutoTokenizer
import pandas as pd
from datasets import Dataset
import torch
torch.set_printoptions(threshold=10000)

In [31]:
data = datasets.load_from_disk("merge")
shuffle_data = data.shuffle()
shuffle_data

Dataset({
    features: ['data'],
    num_rows: 457887
})

In [32]:
def filter_data(sample):
    turns = len(sample['data'])
    prompt_len = sum(len(s) for s in sample['data'])
    if prompt_len > 500 and prompt_len < 4090:
        if turns < 6:
            if random.random() < 0.8 :
                return True
            else:
                return False
        else:
            return True
    else:
        return False
# 应用过滤器
filtered_data = shuffle_data.filter(filter_data)
filtered_data

Filter:   0%|          | 0/457887 [00:00<?, ? examples/s]

Dataset({
    features: ['data'],
    num_rows: 255735
})

In [33]:
system_msg = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n"""
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
# tokenizer.add_special_tokens({'pad_token':'[PAD]'})

In [34]:
chat = []
for item in filtered_data:
    conv = [  {"role": "system", "content": system_msg }]
    for idx, msg in enumerate(item["data"]):
        if idx % 2 ==0:
            conv.append({"role": "user", "content": msg})
        else:
            conv.append({"role": "assistant", "content": msg})
    chat.append(conv)

In [43]:
chat_tokens = tokenizer.apply_chat_template(chat, tokenize=True)

In [49]:
len(chat_tokens[0])

477

In [50]:
tokenizer.decode(chat_tokens[0])

"<s> [INST] <<SYS>>\n\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<</SYS>>\n\nGiven the text: Pay a deposit online now to confirm your spot for summer camps!\n10% sibling discount available - please contact the ITW office after registering to confirm siblings’ names.\nDeposits are non refundable and only transferrable to another holiday course if notice received more than a week in advance of camp start date and if there is space in the camp you wish to move to. Balances of fees are due on or before the first day of camp, p

In [51]:
chat[0]

[{'role': 'system',
  'content': "\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n"},
 {'role': 'user',
  'content': 'Given the text: Pay a deposit online now to confirm your spot for summer camps!\n10% sibling discount available - please contact the ITW office after registering to confirm siblings’ names.\nDeposits are non refundable and only transferrable to another holiday course if notice received more than a week in advance of camp start date and if there is space in the camp you wish to move to. Balances of fees are due on

In [54]:
filter_length_idx = []
for idx, item in enumerate(chat_tokens):
    if len(item) < 1024:
        filter_length_idx.append(idx)

In [56]:
len(filter_length_idx)

255147

In [68]:
select_chat = []
for i in filter_length_idx:
    select_chat.append(chat[i])

In [73]:
select_chat_format = tokenizer.apply_chat_template(select_chat, tokenize=False)

In [78]:
select_chat_format[0]

"<s>[INST] <<SYS>>\n\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<</SYS>>\n\nGiven the text: Pay a deposit online now to confirm your spot for summer camps!\n10% sibling discount available - please contact the ITW office after registering to confirm siblings’ names.\nDeposits are non refundable and only transferrable to another holiday course if notice received more than a week in advance of camp start date and if there is space in the camp you wish to move to. Balances of fees are due on or before the first day of camp, pl

In [59]:
from trl import DataCollatorForCompletionOnlyLM
from datasets import Dataset

In [79]:
df = pd.DataFrame(select_chat_format, columns=["convs"])

In [80]:
instru_data = Dataset.from_pandas(df)

In [82]:
train_test_split =instru_data.train_test_split(test_size=0.2)

In [83]:
train_test_split

DatasetDict({
    train: Dataset({
        features: ['convs'],
        num_rows: 204117
    })
    test: Dataset({
        features: ['convs'],
        num_rows: 51030
    })
})

In [84]:
train_test_split.save_to_disk("train_and_test")

Saving the dataset (0/1 shards):   0%|          | 0/204117 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/51030 [00:00<?, ? examples/s]

In [86]:
train_test_split['train'].save_to_disk("train")

Saving the dataset (0/1 shards):   0%|          | 0/204117 [00:00<?, ? examples/s]

In [87]:
train_test_split['test'].save_to_disk("test")

Saving the dataset (0/1 shards):   0%|          | 0/51030 [00:00<?, ? examples/s]